TODO
Correction for SepReformerSettings sr duplicated

In [1]:
import os
import warnings
import torch

In [2]:
warnings.filterwarnings('ignore')

In [3]:
workdir, _ = os.path.split(os.path.abspath(os.getcwd()))
input = os.path.join(os.getcwd(), 'SepReformer/sample_wav/sample_WSJ.wav')

In [4]:
from simple_vg.vg_types import AudioSetting

settings: AudioSetting = {
    'mono': True,
    'n_fft': None,
    'sr': 16000,
    'use_torch': True,
    'window_size': None,
    'opt_settings': None
}


In [5]:
from simple_vg.process_audio import LoadAudioFile, LoadAudioSettings, LoadAudioBackends

load_settings: LoadAudioSettings = {
    'backend': LoadAudioBackends.TORCH_SOX,
    'effects': [],
    'channel_first': True
}

settings['opt_settings'] = load_settings

load_audio = LoadAudioFile(settings)

In [6]:
from simple_vg.process_audio import NoiseReductionModels, CleaningAudio, CleaningAudioSetting

clean_settings: CleaningAudioSetting = {
    'reduce_noise_lib': NoiseReductionModels.NOISEREDUCE,
    'stationary': False
}
settings['opt_settings'] = clean_settings
clean_audio = CleaningAudio(settings)

In [7]:
from simple_vg.simple_elements import ClosureElement

only_wav = ClosureElement(lambda _, inp: inp[0], None)

In [8]:
from simple_vg.speech_separation.sepreformer import *

sep_model_settings: SepReformerSetting = {
    'batch_size': 1,
    'chunk_max_len': None,
    'distributed_gpu': False,
    'model': SepReformerModels.SepReformer_Base_WSJ0,
    'n_speaker': 2,
    'sr': 16000
}


In [9]:
from simple_vg.process_overlap import *

sep_settings: SeparateOverlappedSettings = {
    'model_settings': sep_model_settings,
    'model': SeparateOverlappedModels.SepReformer
}

overlap_settings: ProcessOverlappedAudioSettings = {
    'tasks': ProcessOverlappedAudioTasks.SEPARATE_AUDIO,
    'separation_settings': sep_settings
}

settings['opt_settings'] = overlap_settings
overlap_elem = ProcessOverlappedAudio(settings)

In [10]:
from simple_vg.activity_detection import *

vad_p_settings: VADPipelineSetting = {
    'vad_model': VADModels.Silero_VAD
}

vad_settings: VoiceActivityDetectionSetting = {
    'pipeline_settings': vad_p_settings,
    'tasks': [
        VoiceActivityDetectionTasks.ACTIVITY_DETECTION, 
        VoiceActivityDetectionTasks.SPLIT_BY_ACTIVITY
    ]
}

settings['opt_settings'] = vad_settings

vad_elem = VoiceActivityDetection(settings)

In [11]:
audio_tensor_squeeze = ClosureElement(lambda _, inp: inp.squeeze(), None)

In [12]:
import simple_vg.extract_text as stt
from simple_vg.extract_text import ExtractText

stt_settings = stt.default_settings()
stt_settings['batch_size'] = 2
settings['opt_settings'] = stt_settings
stt_mod = ExtractText(settings) 
#Input shape: (..., S) [(x, y, z, w, S) => view(x*y*z*w=B_stt, S)]
#Output shape: (B_stt, S) => [[DecodingResult * ceil(S/3000)] * B_stt]


In [13]:
from simple_vg.simple_elements import *

In [14]:
output_audio_paths = [os.path.join(os.getcwd(), 'SepReformer/sample_wav/sample_WSJ_output'+str(i)+'.wav') for i in range(2)]
output_text_data_paths = [os.path.join(os.getcwd(), 'SepReformer/sample_wav/sample_WSJ_output.txt')]

In [15]:
from simple_vg.process_audio import SaveAudioFiles, SaveAudioSettings
from simple_vg.process_text import ToTextDataset, SaveTextFile
settings['opt_settings'] = {}
settings['n_channels'] = 1

In [ ]:
import sys
from simple_vg.pipeline import VGPipeline

dbg = True
def fit_whisper_result_to_elem_input(elem_obj, input):
    output_dicts = []
    for decode_results, audio_path in zip(*input):
        _dict = {}
        script: str = ''
        for result in decode_results:
            _dict['language'] = result.language
            script += result.text
        _dict['text'] = script.replace('\n', ' ')
        _dict['file_path'] = audio_path
        output_dicts.append(_dict)
    return output_dicts

pipeline_elems = [
    load_audio, 
    only_wav,
    # clean_audio, (seems it does not helped for this sample audio.....) 
    overlap_elem,
    ParallelElement(
        [
            [
                audio_tensor_squeeze,
                ClosureElement(lambda _, inp: [inp[i] for i in range(len(inp))], None),
                TensorToDevice(device='cpu')
            ],
            [stt_mod],
            [InputElement(output_audio_paths)]
        ],
        dbg=dbg, name='ProcessAudio'
    ),
    ParallelElement(
        [
            [
                MaskElement([1, 0, 1]), 
                SaveAudioFiles(settings)
            ],
            [
                MaskElement([0, 1, 1]),
                ClosureElement(fit_whisper_result_to_elem_input, None),
                ParallelElement(
                    [
                        [ToTextDataset(['file_path', 'language', 'text'], default=''), ClosureElement(lambda _, inp: [inp], None)],
                        [InputElement(output_text_data_paths)]
                    ], dbg=dbg, name='ProcessText'
                ),
                SaveTextFile()
            ]
        ], dbg=dbg, name='ProcessOutput'
    )
    ]
pipeline = VGPipeline()
pipeline.sequential(pipeline_elems)
pipeline.run(input=input, dbg=dbg)


ValidateElement:	Data: (tensor([[-0.0293, -0.0385, -0.0384,  ..., -0.0384, -0.0499, -0.0310]]), 16000)
Index-0 Element: LoadAudioFile Output: tuple, shape: None
ValidateElement:	Data: tensor([[-0.0293, -0.0385, -0.0384,  ..., -0.0384, -0.0499, -0.0310]])
Index-1 Element: ClosureElement Output: Tensor, shape: torch.Size([1, 147186])
ValidateElement:	Data: tensor([[-0.0053, -0.0056, -0.0047,  ...,  0.0255,  0.0286,  0.0298]],
       device='cuda:0')
Index-2 Element: CleaningAudio Output: Tensor, shape: torch.Size([1, 146944])
ValidateElement:	Data: tensor([[[[-0.0402, -0.0330, -0.0336,  ...,  0.0895,  0.1154,  0.1716]],

         [[ 0.0304,  0.0295,  0.0244,  ..., -0.5869, -0.6393, -0.6792]]]],
       device='cuda:0')
Index-3 Element: ProcessOverlappedAudio Output: Tensor, shape: torch.Size([1, 2, 1, 146944])
ProcessAudio	Root Input: Tensor, shape: torch.Size([1, 2, 1, 146944])
ValidateElement:	Data: tensor([[-0.0402, -0.0330, -0.0336,  ...,  0.0895,  0.1154,  0.1716],
        [ 0.0304, 

[None, None]

In [17]:
clean_audio.ret

{'reduce_noise': tensor([[-0.0053, -0.0056, -0.0047,  ...,  0.0255,  0.0286,  0.0298]],
        device='cuda:0')}